<a href="https://www.quantrocket.com"><img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png"></a>

<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

# Scheduling

To automate trading of the `qval-live` strategy, the remaining step is to schedule data to be collected and Moonshot to run at the appropriate times. Running cron jobs is the task of the `countdown` service. The example commands to run are provided in [quantrocket.countdown.crontab.sh](quantrocket.countdown.crontab.sh).

## Crontab overview

The highlights of the crontab are as follows.

### Data collection

We collect Sharadar fundamentals each weekday in the early morning hours:
```shell
# Collect Sharadar fundamentals overnight 
0 3 * * * mon-fri quantrocket fundamental collect-sharadar  
```

We keep our trading calendar up-to-date by re-collecting it each morning:
```shell
# collect NYSE trading hours each morning (provides trading hours for the next month)
30 8 * * mon-fri quantrocket master collect-calendar -e 'NYSE'
```

And we collect price history from IB each evening if the market was open that day:
```shell
# Collect IB history if market was open
0 17 * * mon-fri quantrocket master isopen 'NYSE' --ago 6h && quantrocket history collect 'nyse-stk-1d' --priority
```

### Live/paper trading

We schedule Moonshot to run on the first trading day of each quarter, mirroring the rebalancing interval of our backtest. As written, the following command will simply log the orders to flightlog; you can alter the command to send the orders to the blotter. You can read more about these commands in the usage guide.

```shell
# Trade qval-live on the first day of the quarter. This command "paper trades" qval-live by logging orders to flightlog; to 
# live or paper trade with IB, send orders to blotter instead:
#     ... quantrocket moonshot trade 'qval-live' | quantrocket blotter order -f -
0 9 * * mon-fri quantrocket master isopen 'NYSE' --in 1h && quantrocket master isclosed 'NYSE' --since 'Q' && quantrocket moonshot trade 'qval-live' | quantrocket flightlog log -n 'quantrocket.moonshot'
```

## Install the crontab

> This section assumes that you're not already using your `countdown` service for any scheduled tasks and that you haven't yet set its timezone. If you're already using `countdown`, you can edit your existing crontab, or add a new countdown service for New York tasks. See the usage guide for help.  

All the commands on the provided crontab are intended to be run in New York time. By default, the countdown timezone is UTC:

In [1]:
from quantrocket.countdown import get_timezone, set_timezone
get_timezone()

{'timezone': 'UTC'}

So first, set the countdown timezone to New York time:

In [2]:
set_timezone("America/New_York")

{'status': 'successfully set timezone to America/New_York'}

Install the crontab by moving it to the `/codeload` directory. (First open a flightlog terminal so you can see if it loads successfully.)

In [3]:
# move file over unless it already exists
![ -e /codeload/quantrocket.countdown.crontab* ] && echo 'oops, the file already exists!' || mv quantrocket.countdown.crontab.sh /codeload/

You should see a success message in flightlog:

```
quantrocket.countdown: INFO Successfully loaded quantrocket.countdown.crontab.sh
```

***

[Back to Introduction](Introduction.ipynb)